In [ ]:
from . import actions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from collections import OrderedDict
from selenium import webdriver


In [ ]:
driver = webdriver.Chrome()

In [ ]:

email = "LINKEDIN EMAIL"
password = "LINKEDIN PWD"
actions.login(driver, email, password)


In [ ]:
# LINKEDIN URL FOR QUERY TO SCRAPE USERS FOR
# PG: NUMBER OF PAGES OF PEOPLE TO SCRAPE FOR

url = 'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101282230%22%5D&keywords=innovation%20manager&network=%5B%22F%22%2C%22S%22%5D&origin=FACETED_SEARCH&sid=l1V'
pg = 2


In [ ]:
driver.get(url)

In [ ]:
db = []

for i in range(0, pg):
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'entity-result__item')))
    cards = driver.find_elements(By.CLASS_NAME, 'entity-result__item')

    for card in cards: 
        try:
            title = card.find_element(By.CLASS_NAME, 'entity-result__title-text').find_element(By.TAG_NAME, "a")
            lnk = title.get_attribute('href')
            name = title.text.split('\n')[0]
            primary = card.find_element(By.CLASS_NAME, 'entity-result__primary-subtitle').text
            secondary = card.find_element(By.CLASS_NAME, 'entity-result__secondary-subtitle').text
            summary = card.find_element(By.CLASS_NAME, 'entity-result__summary').text
            insights = card.find_element(By.CLASS_NAME, 'entity-result__insights')
            connections = insights.find_elements(By.TAG_NAME, "a")
            l_conns = []
            for con in connections:
                con_name = con.text
                con_lnk = con.get_attribute('href')
                l_conns.append([con_name, con_lnk])

            db.append({
                'name': name,
                'lnk': lnk,
                'primary': primary,
                'secondary': secondary,
                'summary': summary,
                'connections': l_conns
            })
        except:
            pass

    if i < (pg-1):
        while True:
            try: 
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination')))
                pages = driver.find_element(By.CLASS_NAME, "artdeco-pagination")

                WebDriverWait(pages, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'artdeco-pagination__button--next'))).click()
                break
            except:
                pass



In [ ]:
df = pd.DataFrame(db)

In [ ]:
df.head()

In [ ]:

experiences = []

urls = df.lnk.values
for url in urls:
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'experience')))
    experience = driver.find_element(By.ID, 'experience')
    experience_list = experience.find_element(By.XPATH, '../div[3]/ul')
    pvs_list = experience_list.find_elements(By.XPATH, '*')

    for i, pvs in enumerate(pvs_list):
        try:

            ps = pvs.find_element(By.XPATH, 'div/div[2]/div[2]/ul')
            c = ps.find_element(By.XPATH, '../../div')
            company_name = c.text
            lnk = c.find_element(By.TAG_NAME, "a").get_attribute('href')
            company_name = list(OrderedDict.fromkeys(company_name.split('\n')))

            ps = ps.find_elements(By.XPATH, '*')
            for n in ps:
                ps_n = n.find_element(By.XPATH, 'div[contains(@class, "pvs-entity")]')

                ps_n = list(OrderedDict.fromkeys(ps_n.text.split('\n')))
                final = [url, company_name[0], lnk] + ps_n
                experiences.append(final)
        except:

            x = pvs.find_element(By.XPATH, 'div/div[2]/div')
            lnk = pvs.find_element(By.XPATH, 'div/div').find_element(By.TAG_NAME, "a").get_attribute('href')
            ps = list(OrderedDict.fromkeys(x.text.split('\n')))

            final = [url, ps[1], lnk, ps[0]]
            if len(ps) >= 4:
                final = final + [ps[2], ps[3]]
            elif len(ps) >= 3:
                final = final + [ps[2]]
            experiences.append(final)

In [ ]:
experiences_df = pd.DataFrame([item[:6] for item in experiences], columns = ['lnk', 'company', 'company_url', 'position', 'duration', 'location'])


In [ ]:
experiences_df = experiences_df.merge(df, on='lnk')


In [ ]:
experiences_df.head()

# Test

AUTOMATE THE SENDING OF PERSONALIZED MESSAGES

In [ ]:
url = 'LINKEDIN USER LINK'
driver.get(url)

In [ ]:
actions = driver.find_element(By.CLASS_NAME, 'pvs-profile-actions')

In [ ]:
btns = actions.find_elements(By.TAG_NAME, 'button')

In [ ]:
for i in btns:
    if i.text.strip() == 'Message':
        i.click()
        break

In [ ]:
driver.find_element(By.CLASS_NAME, 'msg-form__contenteditable').find_element(By.TAG_NAME, 'p').send_keys('TEST MESSAGE')

In [ ]:
driver.find_element(By.CLASS_NAME, 'msg-form__send-button').click()

In [ ]:
driver.quit()